In [1]:
import os
import json
import mne
import scipy

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from mne.preprocessing import ICA
from mne.time_frequency import psd_array_welch
from scipy.signal import hilbert, coherence

try:
    import PyQt5.QtCore
    %matplotlib qt
except ImportError:
    %matplotlib inline

mne.set_log_level('WARNING')

In [2]:
import my_functions as my_fun
import my_visualization_functions as my_vis_fun

# Data collection

In [3]:
# Define subjects to process
subjects = [f"S{str(i).zfill(3)}" for i in range(1, 20)]  
data_path = "eeg-motor-movementimagery-dataset-1.0.0/files/"

In [4]:
# Process EEG data
eeg_data = my_fun.process_eeg(subjects=subjects, data_path=data_path, mode="automatic", apply_ica=False)


🔄 Processing S001...
✅ Extracted 7 epochs for S001 - R03 (real_right_hand)
✅ Extracted 14 rest epochs for S001 - R03
✅ Extracted 7 epochs for S001 - R07 (real_right_hand)
✅ Extracted 14 rest epochs for S001 - R07
✅ Extracted 8 epochs for S001 - R11 (real_right_hand)
✅ Extracted 14 rest epochs for S001 - R11
✅ Extracted 7 epochs for S001 - R04 (imagined_right_hand)
✅ Extracted 14 rest epochs for S001 - R04
✅ Extracted 7 epochs for S001 - R08 (imagined_right_hand)
✅ Extracted 14 rest epochs for S001 - R08
✅ Extracted 8 epochs for S001 - R12 (imagined_right_hand)
✅ Extracted 14 rest epochs for S001 - R12

🔄 Processing S002...
✅ Extracted 7 epochs for S002 - R03 (real_right_hand)
✅ Extracted 14 rest epochs for S002 - R03
✅ Extracted 8 epochs for S002 - R07 (real_right_hand)
✅ Extracted 14 rest epochs for S002 - R07
✅ Extracted 7 epochs for S002 - R11 (real_right_hand)
✅ Extracted 14 rest epochs for S002 - R11
✅ Extracted 8 epochs for S002 - R04 (imagined_right_hand)
✅ Extracted 14 rest ep

## Visualization for one subject

In [ ]:
subject = "S001"  # Change this to inspect other subjects
condition = "real_right_hand"  # Choose from real_left_hand, imagined_left_hand, etc.
epochs = eeg_data[subject][condition][0]

In [ ]:
my_vis_fun.plot_raw_eeg(eeg_data[subject][condition][0], subject, condition)

In [ ]:
my_vis_fun.plot_erd_ers(epochs, subject, condition)

In [ ]:
my_vis_fun.plot_erd_ers(epochs, subject, condition)

# ERD & ERS Analysis

In [ ]:
def plot_group_erd_ers(eeg_data, subjects, conditions, motor_channels=None, fmin=13, fmax=30):
    """
    Plot group-average ERD/ERS curves for real vs. imagined vs. rest conditions.
    """
    if motor_channels is None:
        motor_channels = ['C3', 'C4', 'Cz', 'CP3', 'CP4', 'FC3', 'FC4']

    plt.figure(figsize=(10, 6))

    condition_colors = {
        'real_right_hand': '#3498db',     # blue
        'imagined_right_hand': '#e74c3c', # red
    }

    for condition in conditions:
        all_erd_curves = []

        for subject in subjects:
            if subject not in eeg_data or condition not in eeg_data[subject]:
                continue

            # Merge runs for this subject and condition
            epochs = mne.concatenate_epochs(eeg_data[subject][condition])
            erd_ers, times, freqs = my_fun.compute_erd_ers(epochs, fmin=fmin, fmax=fmax)

            # Filter for motor channels
            available_channels = [ch for ch in motor_channels if ch in epochs.ch_names]
            if len(available_channels) == 0:
                continue

            ch_indices = [epochs.ch_names.index(ch) for ch in available_channels]

            # Compute average over selected channels and freq band
            freq_mask = (freqs >= fmin) & (freqs <= fmax)
            erd_motor = erd_ers[:, ch_indices, :, :]
            erd_motor = erd_motor[:, :, freq_mask, :]
            erd_curve = np.mean(erd_motor, axis=(0, 1, 2))  # mean over epochs, channels, freqs
            all_erd_curves.append(erd_curve)

        if len(all_erd_curves) > 0:
            group_avg_erd = np.mean(all_erd_curves, axis=0)
            plt.plot(times, group_avg_erd, label=condition.replace("_", " ").title(), 
                     color=condition_colors.get(condition, None))

    plt.axvline(0, color='k', linestyle='--', label='Movement Onset')
    plt.xlabel('Time (s)')
    plt.ylabel('ERD/ERS (% Change from Baseline)')
    plt.title('Group ERD/ERS: Real vs Imagined Movement')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()


In [ ]:
def compute_group_erd_ers_metrics(eeg_data, subjects, conditions, motor_channels=None, fmin=13, fmax=30):
    """
    Computes group-level ERD and ERS metrics.
    """
    if motor_channels is None:
        motor_channels = ['C3', 'C4', 'Cz', 'CP3', 'CP4', 'FC3', 'FC4']

    metrics = []

    for condition in conditions:
        all_erd_curves = []

        for subject in subjects:
            if subject not in eeg_data or condition not in eeg_data[subject]:
                continue

            epochs = mne.concatenate_epochs(eeg_data[subject][condition])
            erd_ers, times, freqs = my_fun.compute_erd_ers(epochs, fmin=fmin, fmax=fmax)

            available_channels = [ch for ch in motor_channels if ch in epochs.ch_names]
            if len(available_channels) == 0:
                continue

            ch_indices = [epochs.ch_names.index(ch) for ch in available_channels]
            freq_mask = (freqs >= fmin) & (freqs <= fmax)

            # FIXED slicing:
            erd_motor = erd_ers[:, ch_indices, :, :]      # slice channels first
            erd_motor = erd_motor[:, :, freq_mask, :]     # apply freq_mask second
            erd_curve = np.mean(erd_motor, axis=(0, 1, 2))  # reduce across epochs, channels, freqs

            all_erd_curves.append(erd_curve)

        if len(all_erd_curves) > 0:
            group_avg_erd = np.mean(all_erd_curves, axis=0)

            # Define time windows
            pre_mask = (times >= -1) & (times < 0)      # ERD window
            post_mask = (times >= 0.5) & (times <= 2)   # ERS window

            metrics.append({
                'Condition': condition.replace('_', ' ').title(),
                'ERD Avg (% Change)': np.mean(np.abs(group_avg_erd[pre_mask])),
                'ERD Peak (% Change)': np.min(group_avg_erd[pre_mask]),
                'ERS Avg (% Change)': np.mean(group_avg_erd[post_mask]),
                'ERS Peak (% Change)': np.max(group_avg_erd[post_mask])
            })

    df_metrics = pd.DataFrame(metrics)
    print(df_metrics)
    return df_metrics



In [ ]:
subjects = [f"S{str(i).zfill(3)}" for i in range(1, 20)] 
conditions = ["real_right_hand", "imagined_right_hand"]

In [ ]:
eeg_data = my_fun.process_eeg(subjects=subjects, data_path=data_path, mode="automatic", apply_ica=False)

In [ ]:
plot_group_erd_ers(eeg_data, subjects, conditions, fmin=13, fmax=30)

In [ ]:
df_erd_metrics = compute_group_erd_ers_metrics(eeg_data, subjects, conditions, fmin=13, fmax=30)

# PLV and Coherence Pipeline for one subject

In [ ]:
real_epochs = mne.concatenate_epochs(eeg_data[subject]['real_right_hand'])
imagined_epochs = mne.concatenate_epochs(eeg_data[subject]['imagined_right_hand'])
rest_epochs = mne.concatenate_epochs(eeg_data[subject]['rest'])

In [ ]:
plv_real = my_fun.compute_plv_matrix(real_epochs)
plv_imagined = my_fun.compute_plv_matrix(imagined_epochs)
plv_rest = my_fun.compute_plv_matrix(rest_epochs)

In [ ]:
my_vis_fun.plot_plv_matrix(plv_real, real_epochs.ch_names, title="PLV - Real Right Hand")
my_vis_fun.plot_plv_matrix(plv_imagined, imagined_epochs.ch_names, title="PLV - Imagined Right Hand")
my_vis_fun.plot_plv_matrix(plv_rest, rest_epochs.ch_names, title="PLV - Rest")

In [ ]:
my_vis_fun.plot_plv_difference(plv_real, plv_imagined, real_epochs.ch_names)
my_vis_fun.plot_plv_difference(plv_real, plv_rest, real_epochs.ch_names, title="PLV Difference (Real - Rest)")

In [ ]:
my_vis_fun.plot_motor_plv_difference(plv_real, plv_imagined, real_epochs.ch_names, title="PLV (Motor Cortex) - Real Right Hand")

# PLV and Coherence Pipeline for multiple subjects

In [ ]:
subjects = [f"S{str(i).zfill(3)}" for i in range(1, 10)]  

plv_results = {"real": [], "imagined": [], "rest": []}

eeg_data_all = {}  # <-- Store all subjects

for subject in subjects:
    print(f"\n🚀 Processing {subject}...")
    
    eeg_data = my_fun.process_eeg([subject], data_path="eeg-motor-movementimagery-dataset-1.0.0/files/", mode="automatic", apply_ica=False)

    # Merge subject's data into master dict
    eeg_data_all.update(eeg_data)

    # Compute PLV matrix (optional if you also want group matrix averaging)
    real_epochs = mne.concatenate_epochs(eeg_data[subject]["real_right_hand"])
    imagined_epochs = mne.concatenate_epochs(eeg_data[subject]["imagined_right_hand"])
    rest_epochs = mne.concatenate_epochs(eeg_data[subject]["rest"])

    plv_real = my_fun.compute_plv_matrix(real_epochs)
    plv_imagined = my_fun.compute_plv_matrix(imagined_epochs)
    plv_rest = my_fun.compute_plv_matrix(rest_epochs)

    plv_results["real"].append(plv_real)
    plv_results["imagined"].append(plv_imagined)
    plv_results["rest"].append(plv_rest)


In [ ]:
# Convert to numpy arrays for easy averaging
plv_real_group = np.mean(plv_results["real"], axis=0)
plv_imagined_group = np.mean(plv_results["imagined"], axis=0)
plv_rest_group = np.mean(plv_results["rest"], axis=0)

In [ ]:
my_vis_fun.plot_plv_matrix(plv_real_group, real_epochs.ch_names, title="Group Average PLV - Real Right Hand")
my_vis_fun.plot_plv_matrix(plv_imagined_group, imagined_epochs.ch_names, title="Group Average PLV - Imagined Right Hand")
my_vis_fun.plot_plv_matrix(plv_imagined_group, imagined_epochs.ch_names, title="Group Average PLV - Rest")

my_vis_fun.plot_plv_difference(plv_real_group, plv_imagined_group, real_epochs.ch_names)
my_vis_fun.plot_plv_difference(plv_real_group, plv_rest_group, real_epochs.ch_names, title="PLV Difference (Real - Rest)")

#my_vis_fun.plot_motor_plv_difference(plv_real_group, plv_imagined_group, real_epochs.ch_names)


In [ ]:
# Define subjects and channel pairs
#subjects = ["S001", "S002", "S005"]
conditions = {"real": "real_right_hand", "imagined": "imagined_right_hand", "rest": "rest"} 

channel_pairs = [
    ("C3", "C4"),  # Primary motor cortex
    ("C1", "C2"),  # Central midline
    ("FC3", "FC4"),  # Frontal-motor
    ("Cz", "CPz"),  # Midline execution
    ("CP3", "CP4"),  # Parietal-motor
    ("Fz", "Cz"),  # Frontal-central connection
    ("O1", "O2"),  # Occipital (unrelated)
    ("PO7", "PO8"),  # Parietal (unrelated)
    ("Fp1", "PO8")   # Prefrontal (unrelated)
]

# Analyze
df_plv_coh = my_fun.analyze_pairwise_plv_coherence(subjects, eeg_data_all, conditions, channel_pairs)

# Plot!
my_vis_fun.plot_plv_coherence(df_plv_coh, metric="PLV Mean")
my_vis_fun.plot_plv_coherence(df_plv_coh, metric="Coherence Mean")

In [ ]:
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import multipletests

def paired_ttest_plv(df, metric="PLV Mean", alpha=0.05):
    pairs = df["Channel Pair"].unique()
    results = []

    # Step 1: Collect all p-values for both comparisons
    p_values_real_rest = []
    p_values_real_imagined = []

    for pair in pairs:
        df_pair = df[df["Channel Pair"] == pair]
        pivot_df = df_pair.pivot(index="Subject", columns="Condition", values=metric)
        pivot_df = pivot_df.dropna()

        t_real_rest, p_real_rest = ttest_rel(pivot_df["Real"], pivot_df["Rest"])
        t_real_imagined, p_real_imagined = ttest_rel(pivot_df["Real"], pivot_df["Imagined"])

        results.append({
            "Channel Pair": pair,
            "t Real vs Rest": t_real_rest,
            "p Real vs Rest": p_real_rest,
            "t Real vs Imagined": t_real_imagined,
            "p Real vs Imagined": p_real_imagined
        })

        p_values_real_rest.append(p_real_rest)
        p_values_real_imagined.append(p_real_imagined)

    # Step 2: Apply Bonferroni correction
    _, p_real_rest_corr, _, _ = multipletests(p_values_real_rest, alpha=alpha, method='bonferroni')
    _, p_real_imagined_corr, _, _ = multipletests(p_values_real_imagined, alpha=alpha, method='bonferroni')

    # Step 3: Merge corrected p-values back into results
    for i, res in enumerate(results):
        res["p Real vs Rest (Bonf)"] = p_real_rest_corr[i]
        res["p Real vs Imagined (Bonf)"] = p_real_imagined_corr[i]

    df_stats = pd.DataFrame(results)
    return df_stats


In [ ]:
from scipy.stats import ttest_rel

def paired_ttest_plv(df, metric="PLV Mean"):
    """
    Run paired t-tests for PLV between conditions on each channel pair.
    """
    pairs = df["Channel Pair"].unique()
    results = []

    for pair in pairs:
        df_pair = df[df["Channel Pair"] == pair]
        pivot_df = df_pair.pivot(index="Subject", columns="Condition", values=metric)
        pivot_df = pivot_df.dropna()

        t_real_rest, p_real_rest = ttest_rel(pivot_df["Real"], pivot_df["Rest"])
        t_real_imagined, p_real_imagined = ttest_rel(pivot_df["Real"], pivot_df["Imagined"])

        results.append({
            "Channel Pair": pair,
            "t Real vs Rest": t_real_rest,
            "p Real vs Rest": p_real_rest,
            "t Real vs Imagined": t_real_imagined,
            "p Real vs Imagined": p_real_imagined
        })

    df_stats = pd.DataFrame(results)
    return df_stats


In [ ]:
def report_paired_ttests(df_stats):
    """
    Display t-test results for PLV in a clear and readable way.
    """
    for idx, row in df_stats.iterrows():
        print(f"\n📊 Channel Pair: {row['Channel Pair']}")

        # Real vs Rest
        p1 = row['p Real vs Rest']
        color1 = "\033[92m" if p1 <= 0.05 else "\033[90m"
        print(f"{color1}   Real vs Rest: t = {row['t Real vs Rest']:.2f}, p = {p1:.4f} \033[0m")

        # Real vs Imagined
        p2 = row['p Real vs Imagined']
        color2 = "\033[92m" if p2 <= 0.05 else "\033[90m"
        print(f"{color2}   Real vs Imagined: t = {row['t Real vs Imagined']:.2f}, p = {p2:.4f} \033[0m")


In [ ]:
df_stats = paired_ttest_plv(df_plv_coh, metric="PLV Mean")
report_paired_ttests(df_stats)

# Time frequency plots for key channels (C3, C4)

In [ ]:
from mne.time_frequency import tfr_morlet

# For each condition (real, imagined, rest)
freqs = np.arange(5, 40, 1)  # 5-40 Hz range
n_cycles = freqs / 2

power_real = tfr_morlet(real_epochs, freqs=freqs, n_cycles=n_cycles, return_itc=False)
power_imagined = tfr_morlet(imagined_epochs, freqs=freqs, n_cycles=n_cycles, return_itc=False)
power_rest = tfr_morlet(rest_epochs, freqs=freqs, n_cycles=n_cycles, return_itc=False)

In [ ]:
# Plot time-frequency plots for key channels (C3, C4)
power_real.plot_joint(title='Real Movement', picks=['C3', 'C4'])
power_imagined.plot_joint(title='Imagined Movement', picks=['C3', 'C4'])
power_rest.plot_joint(title='Rest', picks=['C3', 'C4'])

# Decoding PART

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score, GroupKFold
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from mne.decoding import CSP
import seaborn as sns
import mne

In [6]:
def perform_group_csp_lda(X, y, groups, title="Group Classification"):
    """
    Perform CSP+LDA classification with subject-aware cross-validation
    """
    # Define group-stratified cross-validation
    # This ensures that subjects in the test set don't appear in training set
    cv = GroupKFold(n_splits=5)
    
    # Define pipeline
    pipeline = Pipeline([
        ('csp', CSP(n_components=4, reg=None, log=True)),
        ('classifier', LinearDiscriminantAnalysis())
    ])
    
    # Cross-validation
    scores = []
    y_pred = np.zeros_like(y)
    y_prob = np.zeros_like(y, dtype=float)
    
    for train_idx, test_idx in cv.split(X, y, groups):
        pipeline.fit(X[train_idx], y[train_idx])
        
        # Predict
        y_pred[test_idx] = pipeline.predict(X[test_idx])
        
        try:
            y_prob[test_idx] = pipeline.predict_proba(X[test_idx])[:, 1]
        except:
            y_prob[test_idx] = pipeline.decision_function(X[test_idx])
            
        # Calculate fold accuracy
        fold_acc = np.mean(y_pred[test_idx] == y[test_idx])
        scores.append(fold_acc)
    
    # Overall metrics
    mean_score = np.mean(scores)
    std_score = np.std(scores)
    
    print(f"{title}: Accuracy = {mean_score:.3f} ± {std_score:.3f}")
    
    # Confusion matrix
    cm = confusion_matrix(y, y_pred)
    
    # ROC curve metrics
    fpr, tpr, _ = roc_curve(y, y_prob)
    roc_auc = auc(fpr, tpr)
    
    # Train on all data to get CSP patterns
    pipeline.fit(X, y)
    
    return {
        'accuracy': mean_score,
        'std': std_score,
        'confusion_matrix': cm,
        'roc_auc': roc_auc,
        'fpr': fpr,
        'tpr': tpr,
        'pipeline': pipeline
    }

In [7]:
def perform_csp_lda(X, y, title="Classification"):
    """
    Perform CSP+LDA classification
    """
    # Define cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # Define pipeline
    pipeline = Pipeline([
        ('csp', CSP(n_components=4, reg=None, log=True)),
        ('classifier', LinearDiscriminantAnalysis())
    ])
    
    # Cross-validation
    scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')
    mean_score = np.mean(scores)
    std_score = np.std(scores)
    
    print(f"{title}: Accuracy = {mean_score:.3f} ± {std_score:.3f}")
    
    # Predictions for confusion matrix and ROC curve
    y_pred = np.zeros_like(y)
    y_prob = np.zeros_like(y, dtype=float)
    
    for train_idx, test_idx in cv.split(X, y):
        pipeline.fit(X[train_idx], y[train_idx])
        y_pred[test_idx] = pipeline.predict(X[test_idx])
        
        try:
            y_prob[test_idx] = pipeline.predict_proba(X[test_idx])[:, 1]
        except:
            y_prob[test_idx] = pipeline.decision_function(X[test_idx])
    
    # Confusion matrix
    cm = confusion_matrix(y, y_pred)
    
    # ROC curve metrics
    fpr, tpr, _ = roc_curve(y, y_prob)
    roc_auc = auc(fpr, tpr)
    
    # Train on all data to get CSP patterns
    pipeline.fit(X, y)
    
    return {
        'accuracy': mean_score,
        'std': std_score,
        'confusion_matrix': cm,
        'roc_auc': roc_auc,
        'fpr': fpr,
        'tpr': tpr,
        'pipeline': pipeline
    }

In [8]:
def classify_condition_pairs(eeg_data, subjects, freq_band=(8, 30)):
    """
    Perform CSP+LDA classification on multiple condition pairs and return group results.
    
    Parameters:
    -----------
    eeg_data : dict
        Dictionary of EEG data organized by subject and condition
    subjects : list
        List of subject IDs
    freq_band : tuple
        Frequency band for filtering
        
    Returns:
    --------
    group_results : dict
        Dictionary containing group-level results for different condition pairs
    all_individual_results : dict
        Dictionary containing all individual subject results
    """
    # Define condition pairs to analyze
    condition_pairs = [
        ('real_right_hand', 'rest'),
        ('real_right_hand', 'imagined_right_hand')
    ]
    
    # Results dictionary for group analysis
    group_results = {}
    all_individual_results = {}
    
    # Process each condition pair
    for condition_pair in condition_pairs:
        condition1, condition2 = condition_pair
        pair_name = f"{condition1}_vs_{condition2}"
        print(f"\n{'='*80}\nAnalyzing {condition1} vs {condition2}\n{'='*80}")
        
        # Process all subjects for this condition pair
        X_all = []
        y_all = []
        subject_ids = []
        individual_results = {}
        
        for subject in subjects:
            print(f"\nProcessing subject: {subject}")
            
            try:
                # Concatenate epochs for this subject
                if not (condition1 in eeg_data[subject] and condition2 in eeg_data[subject]):
                    print(f"Skipping subject {subject}: missing condition data")
                    continue
                    
                epochs1 = mne.concatenate_epochs(eeg_data[subject][condition1])
                epochs2 = mne.concatenate_epochs(eeg_data[subject][condition2])
                
                # Apply bandpass filter
                fmin, fmax = freq_band
                epochs1 = epochs1.copy().filter(fmin, fmax)
                epochs2 = epochs2.copy().filter(fmin, fmax)
                
                # Select motor channels
                motor_channels = ['C3', 'C4', 'Cz', 'FC3', 'FC4', 'CP3', 'CP4', 'C1', 'C2', 'FC1', 'FC2', 'CP1', 'CP2']
                available_channels = [ch for ch in motor_channels if ch in epochs1.ch_names]
                
                # Pick channels and extract time window of interest (0-2s)
                epochs1_motor = epochs1.copy().pick_channels(available_channels)
                epochs2_motor = epochs2.copy().pick_channels(available_channels)
                
                # Get time indices
                times = epochs1_motor.times
                start_idx = np.where(times >= 0)[0][0]
                end_idx = np.where(times >= 2.0)[0][0] if any(times >= 2.0) else -1
                
                # Extract data
                X1 = epochs1_motor.get_data()[:, :, start_idx:end_idx]  # condition1
                X2 = epochs2_motor.get_data()[:, :, start_idx:end_idx]  # condition2
                
                # Simple undersampling - keep all trials from condition1 (real_right_hand)
                # and subsample condition2 (rest/imagined) to match
                if len(X2) > len(X1):
                    # Randomly sample from condition2 to match condition1 count
                    indices = np.random.choice(len(X2), len(X1), replace=False)
                    X2 = X2[indices]
                    print(f"Subsampled {condition2} to match {condition1}: {len(X1)} trials")
                
                # Create combined dataset for this subject
                X_subject = np.concatenate([X1, X2])
                y_subject = np.concatenate([np.ones(len(X1)), np.zeros(len(X2))])
                
                # Perform individual subject classification
                subject_results = perform_csp_lda(
                    X_subject, 
                    y_subject, 
                    title=f"Subject {subject}: {condition1} vs {condition2}"
                )
                individual_results[subject] = subject_results
                
                # IMPORTANT: Z-score normalization per subject before combining
                # This helps address between-subject variability
                scaler = StandardScaler()
                n_trials, n_channels, n_times = X_subject.shape
                X_subject_reshaped = X_subject.reshape(n_trials, -1)
                X_subject_normalized = scaler.fit_transform(X_subject_reshaped)
                X_subject = X_subject_normalized.reshape(n_trials, n_channels, n_times)
                
                # Add to group dataset
                X_all.append(X_subject)
                y_all.append(y_subject)
                subject_ids.extend([subject] * len(X_subject))
                
            except Exception as e:
                print(f"Error processing subject {subject}: {e}")
                continue
        
        # Store individual results
        all_individual_results[pair_name] = individual_results
        
        # Group-level analysis
        if len(X_all) > 0:
            try:
                X_group = np.concatenate(X_all)
                y_group = np.concatenate(y_all)
                subject_group = np.array(subject_ids)
                
                print(f"\nGroup analysis with {len(X_group)} total trials")
                
                # Store the first subject's info for visualization
                info = None
                for subject in subjects:
                    if subject in eeg_data and len(eeg_data[subject][condition1]) > 0:
                        info = eeg_data[subject][condition1][0].info
                        break
                
                # Perform group classification with subject-aware cross-validation
                group_results[pair_name] = {
                    'X': X_group,
                    'y': y_group,
                    'subjects': subject_group,
                    'info': info,
                    'channels': available_channels,
                    'condition1': condition1,
                    'condition2': condition2,
                    'times': times[start_idx:end_idx],
                    'results': perform_group_csp_lda(X_group, y_group, subject_group, 
                                                    f"Group: {condition1} vs {condition2}")
                }
                
                # Also calculate the average of individual accuracies for comparison
                individual_accuracies = [res['accuracy'] for res in individual_results.values()]
                mean_individual_acc = np.mean(individual_accuracies)
                std_individual_acc = np.std(individual_accuracies)
                group_results[pair_name]['individual_mean_acc'] = mean_individual_acc
                group_results[pair_name]['individual_std_acc'] = std_individual_acc
                
                print(f"Average individual accuracy: {mean_individual_acc:.3f} ± {std_individual_acc:.3f}")
                
            except Exception as e:
                print(f"Error in group analysis: {e}")
        else:
            print("No valid data for group analysis")
    
    return group_results, all_individual_results

In [9]:
def visualize_comparison(group_results, individual_results):
    """
    Visualize and compare classification results between real vs. rest and real vs. imagined
    
    Parameters:
    -----------
    group_results : dict
        Dictionary containing group-level results from classify_condition_pairs
    individual_results : dict
        Dictionary containing individual subject results
    """
    # Check if we have both condition pairs
    pairs = ['real_right_hand_vs_rest', 'real_right_hand_vs_imagined_right_hand']
    if not all(pair in group_results for pair in pairs):
        raise ValueError("Both condition pairs must be in group_results")
    
    # Extract results
    real_vs_rest = group_results['real_right_hand_vs_rest']
    real_vs_imagined = group_results['real_right_hand_vs_imagined_right_hand']
    
    # 1. Create figure with subplots
    fig = plt.figure(figsize=(18, 12))
    
    # 2. Plot ROC curves
    ax1 = plt.subplot2grid((3, 3), (0, 0))
    # Real vs. Rest ROC
    ax1.plot(
        real_vs_rest['results']['fpr'], 
        real_vs_rest['results']['tpr'], 
        lw=2, 
        label=f"Real vs. Rest (AUC = {real_vs_rest['results']['roc_auc']:.2f})"
    )
    # Real vs. Imagined ROC
    ax1.plot(
        real_vs_imagined['results']['fpr'], 
        real_vs_imagined['results']['tpr'], 
        lw=2, 
        label=f"Real vs. Imagined (AUC = {real_vs_imagined['results']['roc_auc']:.2f})"
    )
    ax1.plot([0, 1], [0, 1], 'k--', lw=1)
    ax1.set_xlabel('False Positive Rate')
    ax1.set_ylabel('True Positive Rate')
    ax1.set_title('ROC Curves Comparison')
    ax1.legend(loc="lower right")
    
    # 3. Plot accuracy comparison - BOTH group and individual averages
    ax2 = plt.subplot2grid((3, 3), (0, 1), colspan=2)
    
    # Prepare data
    conditions = ['Real vs. Rest', 'Real vs. Imagined']
    
    # Group accuracies (subject-aware CV)
    group_accuracies = [
        real_vs_rest['results']['accuracy'],
        real_vs_imagined['results']['accuracy']
    ]
    group_errors = [
        real_vs_rest['results']['std'],
        real_vs_imagined['results']['std']
    ]
    
    # Average of individual accuracies
    individual_accuracies = [
        real_vs_rest['individual_mean_acc'],
        real_vs_imagined['individual_mean_acc']
    ]
    individual_errors = [
        real_vs_rest['individual_std_acc'],
        real_vs_imagined['individual_std_acc']
    ]
    
    # Positions for bars
    x = np.arange(len(conditions))
    width = 0.35
    
    # Plot the bars
    ax2.bar(
        x - width/2, 
        group_accuracies, 
        width,
        yerr=group_errors, 
        capsize=10,
        color='#3498db',
        label='Group-Level (Subject-Aware CV)'
    )
    ax2.bar(
        x + width/2, 
        individual_accuracies, 
        width,
        yerr=individual_errors, 
        capsize=10,
        color='#e74c3c',
        label='Average of Individual Subjects'
    )
    
    ax2.axhline(y=0.5, color='r', linestyle='--', alpha=0.7, label='Chance level')
    ax2.set_ylabel('Classification Accuracy')
    ax2.set_title('Accuracy Comparison: Group vs. Individual Average')
    ax2.set_ylim(0.4, 1.0)
    ax2.set_xticks(x)
    ax2.set_xticklabels(conditions)
    ax2.legend()
    
    # 4. Plot confusion matrices side by side
    # Real vs. Rest confusion matrix
    ax3 = plt.subplot2grid((3, 3), (1, 0))
    sns.heatmap(
        real_vs_rest['results']['confusion_matrix'], 
        annot=True, 
        fmt='d', 
        cmap='Blues',
        xticklabels=['Rest', 'Real'],
        yticklabels=['Rest', 'Real'],
        ax=ax3
    )
    ax3.set_xlabel('Predicted Label')
    ax3.set_ylabel('True Label')
    ax3.set_title('Confusion Matrix - Real vs. Rest')
    
    # Real vs. Imagined confusion matrix
    ax4 = plt.subplot2grid((3, 3), (1, 1))
    sns.heatmap(
        real_vs_imagined['results']['confusion_matrix'], 
        annot=True, 
        fmt='d', 
        cmap='Reds',
        xticklabels=['Imagined', 'Real'],
        yticklabels=['Imagined', 'Real'],
        ax=ax4
    )
    ax4.set_xlabel('Predicted Label')
    ax4.set_ylabel('True Label')
    ax4.set_title('Confusion Matrix - Real vs. Imagined')
    
    # 5. Plot CSP patterns for real vs. rest
    ax5 = plt.subplot2grid((3, 3), (1, 2))
    try:
        # Get CSP patterns
        csp_real_rest = real_vs_rest['results']['pipeline'].named_steps['csp']
        info = real_vs_rest['info']
        channels = real_vs_rest['channels']
        
        # Create a copy of the info with only the selected channels
        info_reduced = mne.pick_info(
            info.copy(), 
            [info['ch_names'].index(ch) for ch in channels if ch in info['ch_names']]
        )
        
        # Plot patterns for Real vs. Rest
        pattern_data = csp_real_rest.patterns_[:, 0]
        mne.viz.plot_topomap(
            pattern_data, 
            info_reduced, 
            axes=ax5, 
            show=False
        )
        ax5.set_title('Most Discriminative Pattern\nReal vs. Rest')
    except Exception as e:
        print(f"Error plotting CSP patterns: {e}")
    
    # 6. Plot individual subject accuracies
    ax6 = plt.subplot2grid((3, 3), (2, 0), colspan=3)
    
    # Get individual subject accuracies
    subjects = sorted(individual_results['real_right_hand_vs_rest'].keys())
    real_rest_accs = [individual_results['real_right_hand_vs_rest'][subj]['accuracy'] for subj in subjects]
    real_imag_accs = [individual_results['real_right_hand_vs_imagined_right_hand'][subj]['accuracy'] 
                     for subj in subjects if subj in individual_results['real_right_hand_vs_imagined_right_hand']]
    
    # Ensure we have same number of subjects for both pairs
    if len(real_rest_accs) != len(real_imag_accs):
        min_len = min(len(real_rest_accs), len(real_imag_accs))
        subjects = subjects[:min_len]
        real_rest_accs = real_rest_accs[:min_len]
        real_imag_accs = real_imag_accs[:min_len]
    
    # Set up positions
    x = np.arange(len(subjects))
    width = 0.35
    
    # Plot bars
    ax6.bar(
        x - width/2, 
        real_rest_accs, 
        width,
        color='#3498db',
        label='Real vs. Rest'
    )
    ax6.bar(
        x + width/2, 
        real_imag_accs, 
        width,
        color='#e74c3c',
        label='Real vs. Imagined'
    )
    
    ax6.axhline(y=0.5, color='r', linestyle='--', alpha=0.7, label='Chance level')
    ax6.set_ylabel('Classification Accuracy')
    ax6.set_xlabel('Subject')
    ax6.set_title('Individual Subject Classification Performance')
    ax6.set_xticks(x)
    ax6.set_xticklabels(subjects)
    ax6.legend()
    
    plt.tight_layout()
    plt.suptitle('Group-Level Classification Results Comparison', fontsize=16, y=1.02)
    plt.subplots_adjust(top=0.92)
    plt.show()
    
    # Return summary text
    summary = (
        f"SUMMARY OF RESULTS:\n\n"
        f"Group-Level (Subject-Aware CV):\n"
        f"Real vs. Rest: Accuracy = {real_vs_rest['results']['accuracy']:.3f} ± {real_vs_rest['results']['std']:.3f}, "
        f"AUC = {real_vs_rest['results']['roc_auc']:.3f}\n"
        f"Real vs. Imagined: Accuracy = {real_vs_imagined['results']['accuracy']:.3f} ± {real_vs_imagined['results']['std']:.3f}, "
        f"AUC = {real_vs_imagined['results']['roc_auc']:.3f}\n\n"
        
        f"Average of Individual Subjects:\n"
        f"Real vs. Rest: Accuracy = {real_vs_rest['individual_mean_acc']:.3f} ± {real_vs_rest['individual_std_acc']:.3f}\n"
        f"Real vs. Imagined: Accuracy = {real_vs_imagined['individual_mean_acc']:.3f} ± {real_vs_imagined['individual_std_acc']:.3f}"
    )
    
    print("\n" + "="*80)
    print(summary)
    print("="*80)
    
    return summary

In [10]:
# Usage example:
# 1. Run classification on both condition pairs
group_results, individual_results = classify_condition_pairs(eeg_data, subjects)

# 2. Visualize the comparison
summary = visualize_comparison(group_results, individual_results)


Analyzing real_right_hand vs rest

Processing subject: S001
Subsampled rest to match real_right_hand: 22 trials
Subject S001: real_right_hand vs rest: Accuracy = 0.594 ± 0.105

Processing subject: S002
Subsampled rest to match real_right_hand: 22 trials
Subject S002: real_right_hand vs rest: Accuracy = 0.481 ± 0.173

Processing subject: S003
Subsampled rest to match real_right_hand: 23 trials
Subject S003: real_right_hand vs rest: Accuracy = 0.591 ± 0.144

Processing subject: S004
Subsampled rest to match real_right_hand: 23 trials
Subject S004: real_right_hand vs rest: Accuracy = 0.889 ± 0.070

Processing subject: S005
Subsampled rest to match real_right_hand: 22 trials
Subject S005: real_right_hand vs rest: Accuracy = 0.678 ± 0.102

Processing subject: S006
Subsampled rest to match real_right_hand: 23 trials
Subject S006: real_right_hand vs rest: Accuracy = 0.696 ± 0.177

Processing subject: S007
Subsampled rest to match real_right_hand: 22 trials
Subject S007: real_right_hand vs re

# Decoding Analysis of the frequency bands 

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_frequency_bands(eeg_data, subjects):
    """
    Analyze classification performance across different frequency bands
    within the pre-filtered range of 6-30 Hz.
    """
    # Define frequency bands within the 6-30 Hz range
    frequency_bands = [
        (6, 10),    # Lower alpha
        (8, 12),    # Alpha
        (12, 18),   # Lower beta
        (18, 25),   # Upper beta
        (25, 30),   # Higher beta
        (8, 16),    # Alpha + lower beta
        (16, 30),   # Higher beta
        (6, 30)     # Full band
    ]
    
    # Results storage
    band_results = {'real_vs_rest': [], 'real_vs_imagined': []}
    band_names = [f"{band[0]}-{band[1]}Hz" for band in frequency_bands]
    
    # Analyze each frequency band
    for band in frequency_bands:
        print(f"\n{'-'*50}")
        print(f"Testing frequency band {band[0]}-{band[1]} Hz")
        
        # Run classification with this frequency band
        group_results, _ = classify_condition_pairs(eeg_data, subjects, freq_band=band)
        
        # Store results
        band_results['real_vs_rest'].append(
            group_results['real_right_hand_vs_rest']['individual_mean_acc'])
        band_results['real_vs_imagined'].append(
            group_results['real_right_hand_vs_imagined_right_hand']['individual_mean_acc'])
    
    # Visualize results
    plot_frequency_band_results(band_results, band_names)
    
    return band_results, band_names

def plot_frequency_band_results(band_results, band_names):
    """
    Create a bar plot comparing classification performance across frequency bands.
    """
    plt.figure(figsize=(12, 7))
    
    # Set up positions
    x = np.arange(len(band_names))
    width = 0.35
    
    # Plot bars
    plt.bar(x - width/2, band_results['real_vs_rest'], width, 
            label='Real vs. Rest', color='#3498db')
    plt.bar(x + width/2, band_results['real_vs_imagined'], width, 
            label='Real vs. Imagined', color='#e74c3c')
    
    # Add chance level line
    plt.axhline(y=0.5, color='r', linestyle='--', label='Chance level')
    
    # Labels and formatting
    plt.ylabel('Classification Accuracy')
    plt.xlabel('Frequency Band')
    plt.title('Classification Performance Across Frequency Bands')
    plt.xticks(x, band_names, rotation=45)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    
    # Highlight the best performing bands
    best_real_rest = np.argmax(band_results['real_vs_rest'])
    best_real_imag = np.argmax(band_results['real_vs_imagined'])
    
    plt.annotate(f"Best: {band_results['real_vs_rest'][best_real_rest]:.3f}", 
                xy=(best_real_rest - width/2, band_results['real_vs_rest'][best_real_rest]),
                xytext=(0, 10), textcoords='offset points',
                ha='center', va='bottom', color='#3498db', fontweight='bold')
    
    plt.annotate(f"Best: {band_results['real_vs_imagined'][best_real_imag]:.3f}", 
                xy=(best_real_imag + width/2, band_results['real_vs_imagined'][best_real_imag]),
                xytext=(0, 10), textcoords='offset points',
                ha='center', va='bottom', color='#e74c3c', fontweight='bold')
    
    plt.show()
    
    # Print summary of results
    print("\n" + "="*80)
    print("FREQUENCY BAND ANALYSIS SUMMARY:")
    print("="*80)
    print(f"Best frequency band for Real vs. Rest: {band_names[best_real_rest]} "
          f"({band_results['real_vs_rest'][best_real_rest]:.3f})")
    print(f"Best frequency band for Real vs. Imagined: {band_names[best_real_imag]} "
          f"({band_results['real_vs_imagined'][best_real_imag]:.3f})")
    
    # Print neuroscientific interpretation
    print("\nNEUROSCIENTIFIC INTERPRETATION:")
    interpretation = {
        "6-10Hz": "Lower alpha band, often associated with attentional processes",
        "8-12Hz": "Alpha band, strong association with motor imagery and execution",
        "12-18Hz": "Lower beta band, linked to active motor processing",
        "18-25Hz": "Upper beta band, reflects motor preparation and inhibition",
        "25-30Hz": "Higher beta band, associated with complex cognitive processing",
        "8-16Hz": "Alpha + lower beta, encompasses classical motor rhythm (mu rhythm)",
        "16-30Hz": "Higher beta range, reflects cognitive and integrative aspects",
        "6-30Hz": "Full band, captures all relevant motor-related oscillations"
    }
    
    print(f"Real vs. Rest best in {band_names[best_real_rest]}: {interpretation[band_names[best_real_rest]]}")
    print(f"Real vs. Imagined best in {band_names[best_real_imag]}: {interpretation[band_names[best_real_imag]]}")

# Run the frequency band analysis
band_results, band_names = analyze_frequency_bands(eeg_data, subjects)


--------------------------------------------------
Testing frequency band 6-10 Hz

Analyzing real_right_hand vs rest

Processing subject: S001
Subsampled rest to match real_right_hand: 22 trials
Subject S001: real_right_hand vs rest: Accuracy = 0.475 ± 0.070

Processing subject: S002
Subsampled rest to match real_right_hand: 22 trials
Subject S002: real_right_hand vs rest: Accuracy = 0.528 ± 0.209

Processing subject: S003
Subsampled rest to match real_right_hand: 23 trials
Subject S003: real_right_hand vs rest: Accuracy = 0.516 ± 0.223

Processing subject: S004
Subsampled rest to match real_right_hand: 23 trials
Subject S004: real_right_hand vs rest: Accuracy = 0.911 ± 0.083

Processing subject: S005
Subsampled rest to match real_right_hand: 22 trials
Subject S005: real_right_hand vs rest: Accuracy = 0.403 ± 0.176

Processing subject: S006
Subsampled rest to match real_right_hand: 23 trials
Subject S006: real_right_hand vs rest: Accuracy = 0.547 ± 0.110

Processing subject: S007
Subs

# Decoding analysis of the time windows

In [13]:
def classify_condition_pairs_with_time_window(eeg_data, subjects, time_window=None, freq_band=(8, 30)):
    """
    Perform CSP+LDA classification on multiple condition pairs and return group results.
    
    Parameters:
    -----------
    eeg_data : dict
        Dictionary of EEG data organized by subject and condition
    subjects : list
        List of subject IDs
    freq_band : tuple
        Frequency band for filtering
        
    Returns:
    --------
    group_results : dict
        Dictionary containing group-level results for different condition pairs
    all_individual_results : dict
        Dictionary containing all individual subject results
    """
    # Define condition pairs to analyze
    condition_pairs = [
        ('real_right_hand', 'rest'),
        ('real_right_hand', 'imagined_right_hand')
    ]
    
    # Results dictionary for group analysis
    group_results = {}
    all_individual_results = {}
    
    # Process each condition pair
    for condition_pair in condition_pairs:
        condition1, condition2 = condition_pair
        pair_name = f"{condition1}_vs_{condition2}"
        print(f"\n{'='*80}\nAnalyzing {condition1} vs {condition2}\n{'='*80}")
        
        # Process all subjects for this condition pair
        X_all = []
        y_all = []
        subject_ids = []
        individual_results = {}
        
        for subject in subjects:
            print(f"\nProcessing subject: {subject}")
            
            try:
                # Concatenate epochs for this subject
                if not (condition1 in eeg_data[subject] and condition2 in eeg_data[subject]):
                    print(f"Skipping subject {subject}: missing condition data")
                    continue
                    
                epochs1 = mne.concatenate_epochs(eeg_data[subject][condition1])
                epochs2 = mne.concatenate_epochs(eeg_data[subject][condition2])
                
                # Apply bandpass filter
                fmin, fmax = freq_band
                epochs1 = epochs1.copy().filter(fmin, fmax)
                epochs2 = epochs2.copy().filter(fmin, fmax)
                
                # Select motor channels
                motor_channels = ['C3', 'C4', 'Cz', 'FC3', 'FC4', 'CP3', 'CP4', 'C1', 'C2', 'FC1', 'FC2', 'CP1', 'CP2']
                available_channels = [ch for ch in motor_channels if ch in epochs1.ch_names]
                
                # Pick channels and extract time window of interest (0-2s)
                epochs1_motor = epochs1.copy().pick_channels(available_channels)
                epochs2_motor = epochs2.copy().pick_channels(available_channels)
                
                # Get time indices
                times = epochs1_motor.times
                # NEW - dynamic time window based on argument
                start_idx = np.where(times >= time_window[0])[0][0]
                end_idx = np.where(times >= time_window[1])[0][0] if any(times >= time_window[1]) else -1

                
                # Extract data
                X1 = epochs1_motor.get_data()[:, :, start_idx:end_idx]  # condition1
                X2 = epochs2_motor.get_data()[:, :, start_idx:end_idx]  # condition2
                
                # Simple undersampling - keep all trials from condition1 (real_right_hand)
                # and subsample condition2 (rest/imagined) to match
                if len(X2) > len(X1):
                    # Randomly sample from condition2 to match condition1 count
                    indices = np.random.choice(len(X2), len(X1), replace=False)
                    X2 = X2[indices]
                    print(f"Subsampled {condition2} to match {condition1}: {len(X1)} trials")
                
                # Create combined dataset for this subject
                X_subject = np.concatenate([X1, X2])
                y_subject = np.concatenate([np.ones(len(X1)), np.zeros(len(X2))])
                
                # Perform individual subject classification
                subject_results = perform_csp_lda(
                    X_subject, 
                    y_subject, 
                    title=f"Subject {subject}: {condition1} vs {condition2}"
                )
                individual_results[subject] = subject_results
                
                # IMPORTANT: Z-score normalization per subject before combining
                # This helps address between-subject variability
                scaler = StandardScaler()
                n_trials, n_channels, n_times = X_subject.shape
                X_subject_reshaped = X_subject.reshape(n_trials, -1)
                X_subject_normalized = scaler.fit_transform(X_subject_reshaped)
                X_subject = X_subject_normalized.reshape(n_trials, n_channels, n_times)
                
                # Add to group dataset
                X_all.append(X_subject)
                y_all.append(y_subject)
                subject_ids.extend([subject] * len(X_subject))
                
            except Exception as e:
                print(f"Error processing subject {subject}: {e}")
                continue
        
        # Store individual results
        all_individual_results[pair_name] = individual_results
        
        # Group-level analysis
        if len(X_all) > 0:
            try:
                X_group = np.concatenate(X_all)
                y_group = np.concatenate(y_all)
                subject_group = np.array(subject_ids)
                
                print(f"\nGroup analysis with {len(X_group)} total trials")
                
                # Store the first subject's info for visualization
                info = None
                for subject in subjects:
                    if subject in eeg_data and len(eeg_data[subject][condition1]) > 0:
                        info = eeg_data[subject][condition1][0].info
                        break
                
                # Perform group classification with subject-aware cross-validation
                group_results[pair_name] = {
                    'X': X_group,
                    'y': y_group,
                    'subjects': subject_group,
                    'info': info,
                    'channels': available_channels,
                    'condition1': condition1,
                    'condition2': condition2,
                    'times': times[start_idx:end_idx],
                    'results': perform_group_csp_lda(X_group, y_group, subject_group, 
                                                    f"Group: {condition1} vs {condition2}")
                }
                
                # Also calculate the average of individual accuracies for comparison
                individual_accuracies = [res['accuracy'] for res in individual_results.values()]
                mean_individual_acc = np.mean(individual_accuracies)
                std_individual_acc = np.std(individual_accuracies)
                group_results[pair_name]['individual_mean_acc'] = mean_individual_acc
                group_results[pair_name]['individual_std_acc'] = std_individual_acc
                
                print(f"Average individual accuracy: {mean_individual_acc:.3f} ± {std_individual_acc:.3f}")
                
            except Exception as e:
                print(f"Error in group analysis: {e}")
        else:
            print("No valid data for group analysis")
    
    return group_results, all_individual_results

In [14]:
def analyze_time_windows(eeg_data, subjects):
    """
    Analyze classification performance across different time windows
    within the epoch range from -1 to 3 seconds.
    """
    # Define time windows to analyze
    time_windows = [
        (-1.0, 0.0),    # Baseline/preparation
        (0.0, 0.5),     # Early execution
        (0.5, 1.5),     # Mid execution
        (1.5, 3.0),     # Late execution/recovery
        (0.0, 3.0),     # Full execution period
        (-1.0, 3.0)     # Entire epoch
    ]
    
    # Results storage
    window_results = {'real_vs_rest': [], 'real_vs_imagined': []}
    window_names = [f"{win[0]:.1f}-{win[1]:.1f}s" for win in time_windows]
    
    # Analyze each time window
    for time_window in time_windows:
        print(f"\n{'-'*50}")
        print(f"Testing time window {time_window[0]:.1f}-{time_window[1]:.1f} seconds")
        
        # Run classification with this time window
        group_results, _ = classify_condition_pairs_with_time_window(
            eeg_data, subjects, time_window=time_window)
        
        # Store results
        window_results['real_vs_rest'].append(
            group_results['real_right_hand_vs_rest']['individual_mean_acc'])
        window_results['real_vs_imagined'].append(
            group_results['real_right_hand_vs_imagined_right_hand']['individual_mean_acc'])
    
    # Visualize results
    plot_time_window_results(window_results, window_names)
    
    return window_results, window_names


def plot_time_window_results(window_results, window_names):
    """
    Create a line plot showing how classification performance changes across time windows.
    """
    plt.figure(figsize=(12, 7))
    
    # Set up positions
    x = np.arange(len(window_names))
    
    # Plot lines with markers
    plt.plot(x, window_results['real_vs_rest'], 'o-', 
             linewidth=2, markersize=8, label='Real vs. Rest', color='#3498db')
    plt.plot(x, window_results['real_vs_imagined'], 's-', 
             linewidth=2, markersize=8, label='Real vs. Imagined', color='#e74c3c')
    
    # Add chance level line
    plt.axhline(y=0.5, color='r', linestyle='--', label='Chance level')
    
    # Labels and formatting
    plt.ylabel('Classification Accuracy')
    plt.xlabel('Time Window (seconds)')
    plt.title('Classification Performance Across Time Windows')
    plt.xticks(x, window_names, rotation=45)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Add vertical line at movement onset (t=0)
    plt.axvline(x=2, color='green', linestyle=':', 
                label='Movement Onset', alpha=0.7)
    
    plt.tight_layout()
    
    # Highlight the best performing time windows
    best_real_rest = np.argmax(window_results['real_vs_rest'])
    best_real_imag = np.argmax(window_results['real_vs_imagined'])
    
    plt.annotate(f"Best: {window_results['real_vs_rest'][best_real_rest]:.3f}", 
                xy=(best_real_rest, window_results['real_vs_rest'][best_real_rest]),
                xytext=(0, 10), textcoords='offset points',
                ha='center', va='bottom', color='#3498db', fontweight='bold')
    
    plt.annotate(f"Best: {window_results['real_vs_imagined'][best_real_imag]:.3f}", 
                xy=(best_real_imag, window_results['real_vs_imagined'][best_real_imag]),
                xytext=(0, 10), textcoords='offset points',
                ha='center', va='bottom', color='#e74c3c', fontweight='bold')
    
    plt.show()
    
    # Print summary of results
    print("\n" + "="*80)
    print("TIME WINDOW ANALYSIS SUMMARY:")
    print("="*80)
    print(f"Best time window for Real vs. Rest: {window_names[best_real_rest]} "
          f"({window_results['real_vs_rest'][best_real_rest]:.3f})")
    print(f"Best time window for Real vs. Imagined: {window_names[best_real_imag]} "
          f"({window_results['real_vs_imagined'][best_real_imag]:.3f})")
    
    # Print neuroscientific interpretation
    print("\nNEUROSCIENTIFIC INTERPRETATION:")
    interpretation = {
        "-1.0-0.0s": "Preparation phase, anticipatory activity",
        "0.0-0.5s": "Early movement execution, movement initiation",
        "0.5-1.5s": "Sustained movement, active motor control",
        "1.5-3.0s": "Late phase, potentially includes termination and recovery",
        "0.0-3.0s": "Entire execution phase",
        "-1.0-3.0s": "Full trial including preparation and execution"
    }
    
    print(f"Real vs. Rest best in {window_names[best_real_rest]}: {interpretation[window_names[best_real_rest]]}")
    print(f"Real vs. Imagined best in {window_names[best_real_imag]}: {interpretation[window_names[best_real_imag]]}")

# Run the time window analysis
window_results, window_names = analyze_time_windows(eeg_data, subjects)


--------------------------------------------------
Testing time window -1.0-0.0 seconds

Analyzing real_right_hand vs rest

Processing subject: S001
Subsampled rest to match real_right_hand: 22 trials
Subject S001: real_right_hand vs rest: Accuracy = 0.658 ± 0.017

Processing subject: S002
Subsampled rest to match real_right_hand: 22 trials
Subject S002: real_right_hand vs rest: Accuracy = 0.817 ± 0.153

Processing subject: S003
Subsampled rest to match real_right_hand: 23 trials
Subject S003: real_right_hand vs rest: Accuracy = 0.582 ± 0.183

Processing subject: S004
Subsampled rest to match real_right_hand: 23 trials
Subject S004: real_right_hand vs rest: Accuracy = 0.762 ± 0.189

Processing subject: S005
Subsampled rest to match real_right_hand: 22 trials
Subject S005: real_right_hand vs rest: Accuracy = 0.750 ± 0.129

Processing subject: S006
Subsampled rest to match real_right_hand: 23 trials
Subject S006: real_right_hand vs rest: Accuracy = 0.722 ± 0.131

Processing subject: S00